In [1]:

import ipympl

from lsst.pipe.tasks.sourceClassification import SourceClassificationTask
import lsst.afw.table as afwTable
import lsst.afw.geom as afwGeom
import glob
import sncosmo
import pickle
from astropy.table import Table
import re
import astropy.coordinates as coord
import astropy.units as u

import numpy as np
import glob

import Utils.DiaSourceTools as DSTools
import Utils.SourcePlotting as SP
import matplotlib.pyplot as plt
import matplotlib
from IPython.display import clear_output
import lsst.daf.persistence as dafPersist


In [2]:

DATADIR="/renoir_data_02/jpreyes/lsst_data/CFHTLS_master/"
butler = dafPersist.Butler(DATADIR+"/output") 

filter = 'r'
cat_ids = butler.queryMetadata("deepDiff_class_diaSrc", format=["ccd","filter", "visit"],dataId={"filter":filter})
        
season_catalogs = []
data_refs = []
for c_id in cat_ids:
    if c_id[0] == 14:
        if  butler.datasetExists("deepDiff_class_diaSrc", {"ccd":c_id[0],"filter":c_id[1], "visit":c_id[2]}):
            data_refs.append({"ccd":c_id[0],"filter":c_id[1], "visit":c_id[2]})
            season_catalogs.append(butler.get("deepDiff_class_diaSrc", {"ccd":c_id[0],"filter":c_id[1], "visit":c_id[2]}))

In [3]:
len(season_catalogs)

58

In [4]:
def multi_match_catalogs(catalog_list, data_refs):
    multi_matches = None

    for catalog, data_ref in zip(catalog_list, data_refs):

            if multi_matches is None and len(catalog)>0:
                multi_matches = afwTable.MultiMatch(catalog[0].schema, {'visit':int, 'ccd':int}, radius=afwGeom.Angle(1./3600., afwGeom.degrees))
            if multi_matches is not None:
                multi_matches.add(catalog, {'visit':data_ref["visit"], 'ccd':data_ref["ccd"]})

    results = multi_matches.finish(removeAmbiguous=False)  
    return results



In [5]:
results = multi_match_catalogs(season_catalogs, data_refs)
print len(results)

207366


In [6]:
reload(DSTools)

<module 'Utils.DiaSourceTools' from 'Utils/DiaSourceTools.pyc'>

In [7]:
light_curves=DSTools.get_light_curves_from_multimatch_results2(results)
print len(light_curves)

101213


In [8]:
t_light_curves = DSTools.threshold_light_curves(light_curves, 10)
print len(t_light_curves)

276


In [21]:
t_light_curves = DSTools.threshold_light_curves(light_curves, 1)
print len(t_light_curves)

101213


In [ ]:
lcs1 = []
for t_light_curve in t_light_curves:
    lcs1.append(DSTools.build_lightcurve(t_light_curve, "base_CircularApertureFlux_3_0_flux"))

In [10]:
lcs2 = []
for t_light_curve in t_light_curves:
    lcs2.append(DSTools.build_lightcurve(t_light_curve, "base_CircularApertureFlux_4_5_flux"))

In [11]:
lcs3 = []
for t_light_curve in t_light_curves:
    lcs3.append(DSTools.build_lightcurve(t_light_curve, "ip_diffim_PsfDipoleFlux_pos_flux"))

In [12]:


classification_dict = {0:"Dipole type I", 1: "Dipole type II", 3:"Fringe", 4: "Artifact", 5:"Negative", 6:"Positive", 7:"Point Positive"}
SP.plot_object_distribution(lcs1, classification_dict)

In [13]:
import re
import astropy.coordinates as coord
import astropy.units as u

filter='r'

f = open('/renoir_data_02/jpreyes/lsst_data/sn_control/J_A+A_523_A7_table9.dat.txt','r')
data_elems = f.read()
elems = re.findall('^(06D3.*?)\\|(.*?)\\|', data_elems, re.MULTILINE)

f = open('/renoir_data_02/jpreyes/lsst_data/sn_control/J_A+A_523_A7_table10.dat.txt','r')
data = f.read()

snls_array = []
for sn in elems:
    c = coord.SkyCoord(sn[-1], unit=(u.hourangle, u.deg))
    
    m = re.findall('^'+str(sn[0])+'\\|('+filter+')\\|(.*?)\\|(.*?)\\|(.*?)$', data, re.MULTILINE)

    snls_lc = DSTools.build_light_curve_from_snls_file(m, c)
    plt.errorbar(snls_lc['mjd'], snls_lc['flux'], yerr=snls_lc['flux_error'], fmt='.', color='blue')
    snls_array.append(snls_lc)
    plt.show()

In [14]:
matches = DSTools.match_control_group(lcs1,snls_array)
float(len(matches))/float(len(snls_array))

0.058823529411764705

In [15]:
import lsst.daf.persistence as dafPersist

DATADIR="/renoir_data_02/jpreyes/lsst_data/CFHTLS_master/"
butler = dafPersist.Butler(DATADIR+"/output") 


In [16]:
import lsst.daf.persistence as dafPersist

DATADIR="/renoir_data_02/jpreyes/lsst_data/CFHTLS_master/"
butler = dafPersist.Butler(DATADIR+"/output") 

In [17]:
def get_stamps_in_lc(lc):
    ccds= range(0,36)
    stamps = []
    for point in lc:

        for ccd in ccds: 
            if butler.datasetExists("deepDiff_differenceExp",dataId={"filter":'r', 'visit':point["mjd"], 'ccd':ccd}):
                diffExp = butler.get("deepDiff_differenceExp", dataId={"filter":'r', 'visit':point["mjd"], 'ccd':ccd})
                box = diffExp.getBBox()
                wcs = diffExp.getWcs()
                coord = afwGeom.Point2I(wcs.skyToPixel(point["ra"], point["dec"]))
                if box.contains(coord):
                    break


        #calexp = butler.get("calexp", dataId={"filter":'r', 'visit':point["mjd"], 'ccd':ccd})
        calexp = butler.get("calexp", dataId={"filter":'r', 'visit':point["mjd"], 'ccd':ccd})



        Center = afwGeom.Point2I(wcs.skyToPixel(point["ra"], point["dec"]))
        Begin = afwGeom.Point2I(Center.getX() - 10, Center.getY() - 10)
        End = afwGeom.Point2I(Center.getX() + 10, Center.getY() + 10)

        newBox = afwGeom.Box2I(Begin,End) 
        try:
            cal_stamp = calexp.Factory(calexp, newBox, True)
            stamp = diffExp.Factory(diffExp,newBox, True)
            stamps.append((stamp, cal_stamp))
        except Exception, e:
            print e
            stamps.append((None, None))
            pass
        
    return stamps
    
def get_stamps_in_lc2(lc):
    ccds= range(0,36)
    stamps = []
    
    for point in lc:

     

        diffExp = butler.get("deepDiff_differenceExp", dataId={"filter":'r', 'visit':point["visit"], 'ccd':point["ccd"]})
        calexp = butler.get("calexp", dataId={"filter":'r', 'visit':point["visit"], 'ccd':point["ccd"]})
        box = diffExp.getBBox()
        wcs = diffExp.getWcs()
        coord = afwGeom.Point2I(wcs.skyToPixel(point["coord_ra"], point["coord_dec"]))


        Center = afwGeom.Point2I(wcs.skyToPixel(point["coord_ra"], point["coord_dec"]))
        Begin = afwGeom.Point2I(Center.getX() - 10, Center.getY() - 10)
        End = afwGeom.Point2I(Center.getX() + 10, Center.getY() + 10)

        newBox = afwGeom.Box2I(Begin,End) 
        try:
            cal_stamp = calexp.Factory(calexp, newBox, True)
            stamp = diffExp.Factory(diffExp,newBox, True)
            stamps.append((stamp, cal_stamp))
        except Exception, e:
            print e
            stamps.append((None, None))
            pass
        
    return stamps

def get_stamps_in_lc3(lc, ra, dec):
    ccds= range(0,36)
    stamps = []
    
    for point in lc:

     

        diffExp = butler.get("deepDiff_differenceExp", dataId={"filter":'r', 'visit':point["visit"], 'ccd':point["ccd"]})
        calexp = butler.get("calexp", dataId={"filter":'r', 'visit':point["visit"], 'ccd':point["ccd"]})
        matchexp = butler.get("deepDiff_matchedExp", dataId={"filter":'r', 'visit':point["visit"], 'ccd':point["ccd"]})
        
        box = diffExp.getBBox()
        wcs = diffExp.getWcs()
        coord = afwGeom.Point2I(wcs.skyToPixel(ra, dec))


        Center = afwGeom.Point2I(wcs.skyToPixel(ra, dec))
        Begin = afwGeom.Point2I(Center.getX() - 10, Center.getY() - 10)
        End = afwGeom.Point2I(Center.getX() + 10, Center.getY() + 10)

        newBox = afwGeom.Box2I(Begin,End) 
        try:
            cal_stamp = calexp.Factory(calexp, newBox, True)
            stamp = diffExp.Factory(diffExp,newBox, True)
            mat_stamp = matchexp.Factory(matchexp, newBox, True)
            stamps.append((stamp, cal_stamp, mat_stamp))
        except Exception, e:
            print e
            stamps.append((None, None, None))
            pass
        
    return stamps
    

In [18]:
ind = 10
lc = lcs1[ind]
t_lc = t_light_curves[ind]

stamps = get_stamps_in_lc3(t_lc, np.mean(lc["ra"]), np.mean(lc["dec"]))

In [19]:
plt.close("all")
fig = plt.figure(figsize=(10,5))
fig2 = plt.figure(figsize=(10,10))
plt.figure(fig.number)
#plt.ylim(ymin=-75000, ymax=100000)
tolerance = 5 # points
plt.errorbar(lc["mjd"],lc["flux"], yerr=lc["flux_error"],fmt='.', color='blue', picker=tolerance)

fig.canvas.mpl_connect('pick_event', lambda event: on_pick(event, stamps , fig2.number))
plt.show()

In [20]:
def on_pick(event, stamps, fig_number):
    
    clear_output()
    
    plt.figure(fig_number)
    plt.clf()
    artist = event.artist
    
    xmouse, ymouse = event.mouseevent.xdata, event.mouseevent.ydata
    x, y = artist.get_xdata(), artist.get_ydata()
    ind = event.ind

    stamp, c_stamp, m_stamp = stamps[ind[0]]
    
    #matchexp
    plt.subplot(2,3,3)
    img_arr, mask_arr, var_arr = m_stamp.getMaskedImage().getArrays()
    w,h = m_stamp.getWidth(), m_stamp.getHeight()
    plt.imshow(img_arr, origin='lower', vmin=img_arr.min(), vmax=img_arr.max(), cmap='gray', extent=(0,w-1, 0, h-1), interpolation='none')
    
    plt.subplot(2,1,2) 
    plt.hist(img_arr.flat, 100, alpha=0.5, label="matchexp")

    #difference image
    plt.subplot(2,3,1)
    img_arr, mask_arr, var_arr = stamp.getMaskedImage().getArrays()
    w,h = stamp.getWidth(), stamp.getHeight()  
    plt.imshow(img_arr, origin='lower', vmin=img_arr.min(), vmax=img_arr.max(), cmap='gray', extent=(0,w-1, 0, h-1), interpolation='none')
   
    plt.subplot(2,1,2)
    plt.hist(img_arr.flat, 100, alpha=0.5, label="diffExp")
    
    #calexp
    plt.subplot(2,3,2)
    img_arr, mask_arr, var_arr = c_stamp.getMaskedImage().getArrays()
    w,h = c_stamp.getWidth(), c_stamp.getHeight()
    plt.imshow(img_arr, origin='lower', vmin=img_arr.min(), vmax=img_arr.max(), cmap='gray', extent=(0,w-1, 0, h-1), interpolation='none')
    
    plt.subplot(2,1,2) 
    plt.hist(img_arr.flat, 100, alpha=0.5, label="calexp")
    
    

    
    
    plt.legend(loc='upper right')
    plt.show()
    

In [106]:
plt.subplot(2,3,1)
plt.plot(range(10))
plt.subplot(2,3,2)
plt.plot(range(10))
plt.subplot(2,3,3)
plt.plot(range(10))

plt.subplot(2,1,2)
plt.plot(range(10))
plt.show()

In [ ]:
stamp, c_stamp = stamps[0]
img_arr, mask_arr, var_arr = stamp.getMaskedImage().getArrays()

In [ ]:
plt.hist(img_arr.flat, 100)
plt.show()

In [111]:
img_arr, mask_arr, var_arr = c_stamp.getMaskedImage().getArrays()
plt.hist(img_arr.flat, 100)
plt.show()

In [17]:
plt.close("all")